In [1]:
#importing the libraries
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings
warnings.filterwarnings("ignore")
import nltk
from thefuzz import fuzz
from thefuzz import process

In [2]:
%%time
#loading the data

df=pd.read_csv('wiki_movie_plots_deduped_cleaned_genre_and_plot.csv')
df

Wall time: 1.29 s


,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Genre_updated,Wiki Page,Plot,Plot_cleanned
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,NaN,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr...",bartender working saloon serving drink custome...
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,NaN,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov...",moon painted smiling face hang park night youn...
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,NaN,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed...",film minute long composed two shot first girl ...
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,NaN,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...,lasting second consisting two shot first shot ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,NaN,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...,earliest known adaptation classic fairytale fi...
...,...,...,...,...,...,...,...,...,...,...
42530,2014,The Water Diviner,Turkish,Director: Russell Crowe,Director: Russell Crowe\r\nCast: Russell Crowe...,unknown,NaN,https://en.wikipedia.org/wiki/The_Water_Diviner,"The film begins in 1919, just after World War ...",film begin wxyz world war ended centre around ...
42531,2017,Çalgı Çengi İkimiz,Turkish,Selçuk Aydemir,"Ahmet Kural, Murat Cemcir",comedy,comedy,https://en.wikipedia.org/wiki/%C3%87alg%C4%B1_...,"Two musicians, Salih and Gürkan, described the...",two musician salih gürkan described adventure ...
42532,2017,Olanlar Oldu,Turkish,Hakan Algül,"Ata Demirer, Tuvana Türkay, Ülkü Duru",comedy,comedy,https://en.wikipedia.org/wiki/Olanlar_Oldu,"Zafer, a sailor living with his mother Döndü i...",zafer sailor living mother döndü coastal villa...
42533,2017,Non-Transferable,Turkish,Brendan Bradley,"YouTubers Shanna Malcolm, Shira Lazar, Sara Fl...",romantic comedy,comedy,https://en.wikipedia.org/wiki/Non-Transferable...,The film centres around a young woman named Am...,film centre around young woman named amy tyler...


In [3]:
df['Cast'].nunique()

32182

In [4]:
df['Cast'] = df['Cast'].replace(np.nan, '')

In [5]:
def cast(data):
    df1 = pd.DataFrame(columns = ['Release Year', 'Title', 'Origin/Ethnicity', 'Director', 'Cast','Genre', 'Genre_updated', 'Wiki Page', 'Plot', 'Plot_cleanned' , 'Cast_1'])
    
    for i,row in data.iterrows():
        if len(row['Cast'].split(','))>1:
            for i in row['Cast'].split(','):
               # print(i)
                df=pd.DataFrame(row).T
                df['Cast_1']=i
                df1=pd.concat([df1,df])
        else:
            df=pd.DataFrame(row).T
            df['Cast_1']=df['Cast']
            df1=pd.concat([df1,df])
    df['Cast_1']=df['Cast_1'].apply(lambda x: x.strip())
    return df1.reset_index(drop=True)

In [6]:
%%time
df=cast(df)

Wall time: 20min 4s


In [7]:
df.drop_duplicates(inplace=True)

In [8]:
df

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Genre_updated,Wiki Page,Plot,Plot_cleanned,Cast_1
0,1901,Kansas Saloon Smashers,American,Unknown,,unknown,NaN,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr...",bartender working saloon serving drink custome...,
1,1901,Love by the Light of the Moon,American,Unknown,,unknown,NaN,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov...",moon painted smiling face hang park night youn...,
2,1901,The Martyred Presidents,American,Unknown,,unknown,NaN,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed...",film minute long composed two shot first girl ...,
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,,unknown,NaN,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...,lasting second consisting two shot first shot ...,
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",,unknown,NaN,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...,earliest known adaptation classic fairytale fi...,
...,...,...,...,...,...,...,...,...,...,...,...
151387,2017,Non-Transferable,Turkish,Brendan Bradley,"YouTubers Shanna Malcolm, Shira Lazar, Sara Fl...",romantic comedy,comedy,https://en.wikipedia.org/wiki/Non-Transferable...,The film centres around a young woman named Am...,film centre around young woman named amy tyler...,Sara Fletcher and Ashley Clements
151388,2017,İstanbul Kırmızısı,Turkish,Ferzan Özpetek,"Halit Ergenç, Tuba Büyüküstün, Mehmet Günsür, ...",romantic,romance,https://en.wikipedia.org/wiki/%C4%B0stanbul_K%...,The writer Orhan Şahin returns to İstanbul aft...,writer orhan şahin return i̇stanbul many year ...,Halit Ergenç
151389,2017,İstanbul Kırmızısı,Turkish,Ferzan Özpetek,"Halit Ergenç, Tuba Büyüküstün, Mehmet Günsür, ...",romantic,romance,https://en.wikipedia.org/wiki/%C4%B0stanbul_K%...,The writer Orhan Şahin returns to İstanbul aft...,writer orhan şahin return i̇stanbul many year ...,Tuba Büyüküstün
151390,2017,İstanbul Kırmızısı,Turkish,Ferzan Özpetek,"Halit Ergenç, Tuba Büyüküstün, Mehmet Günsür, ...",romantic,romance,https://en.wikipedia.org/wiki/%C4%B0stanbul_K%...,The writer Orhan Şahin returns to İstanbul aft...,writer orhan şahin return i̇stanbul many year ...,Mehmet Günsür


In [9]:
%%time
list_dir=[[i,df[df['Cast_1']==i]['Genre_updated'].nunique()] for i in df['Cast_1'].unique()]

Wall time: 5min 28s


In [14]:

most=pd.DataFrame(list_dir).sort_values(by=[1],ascending=False)[:10]
most

,0,1
0,,27
595,Jr.,20
4285,Christopher Plummer,16
26385,Amitabh Bachchan,15
5085,Donald Sutherland,15
8251,Johnny Depp,15
5369,Susan Sarandon,15
9760,Penélope Cruz,14
7361,Jackie Chan,14
5946,Michael Caine,14


### Most versatile 
actor: Christopher Plummer

actress: Susan Sarandon
